In [ ]:
## General data processing and visualisation use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import os
import glob
import plotly.express as px

## For webscraping
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

## Machine learning / Deep learning classification models
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import tree
from sklearn.metrics import ConfusionMatrixDisplay

## XGBoost as extra
import xgboost as xgb

## Set the pandas display option set to max_columns
pd.set_option('display.max_columns', None)

## Natural language processing
from collections import Counter
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import spacy


In [ ]:
df_n_fresh = pd.read_csv('avalanche_types.csv')
df_n_fresh.drop(columns=['Unnamed: 0'], inplace=True)
df_n_fresh.head()

In [ ]:
'''
Function to calculate the amoung of NULLs in a dataframe per its given columns
--- INPUT: pandas dataframe
--- OUTPUT: pandas dataframe
'''

def calc_null(dataframe):

    # Initialise a new df
    missing_df = pd.DataFrame(df.isna().sum(), columns=['missing_vals'])
    missing_df['percentage'] = 0.0

    # Initialise math and loop var
    total_count = len(dataframe)

    # Assign percentage
    missing_df['percentage'] = round(missing_df['missing_vals'] / total_count * 100, 3)

    # Sort accordingly
    missing_df = missing_df.sort_values('percentage', ascending=False)

    return missing_df

In [ ]:
def year_extraction(x):
    x = x.split(' ')
    return x[3]

def month_extraction(x):
    x = x.split(' ')
    return x[1]

df_n_fresh['year'] = df_n_fresh['dates'].apply(year_extraction)
df_n_fresh['month'] = df_n_fresh['dates'].apply(month_extraction)

## EDA (01)

In [ ]:
''' Shows the missing percentage df '''

# Use previously created function to check amount of missing values and %
df_missing_original = calc_null(df)

# Reset the index
df_missing_original_disp = df_missing_original.reset_index()

# Initialise the figure and run seaborn for visualisations.
plt.figure(figsize = (6, 6))

sns.barplot(
    data = df_missing_original_disp,
    x = 'index',
    y = 'percentage'
)
plt.xticks(
    rotation=45,
    horizontalalignment='right',
    fontweight ='light',
    fontsize ='large'
)
plt.title('percentage of nulls per column')
plt.show()

In [ ]:
''' Unique display of avalanches per given feature '''

u_df = df.describe().loc['unique'].copy()
temp_df = pd.DataFrame({'unique_count':u_df})
temp_df = temp_df.sort_values('unique_count', ascending=False)
values = temp_df.unique_count
plt.figure(figsize=(5,5))
sns.barplot(
    data=temp_df,
    x=temp_df.index,
    y='unique_count'
)
plt.xticks(
    rotation=45,
    horizontalalignment = 'right',
    fontweight = 'light',
    fontsize = 'large'
)
plt.title('unique counts per column')
# plt.savefig('Y/N_avalanches_region.png', dpi=300, bbox_inches='tight')
plt.show()
print(values)

In [ ]:
# Avalanche_Y/N will not have any nulls as been web scraped.
disp_df = df_n_fresh.groupby('region')[['avalanche_Y/N']].value_counts().reset_index().copy()
disp_df = disp_df.rename(columns={ disp_df.columns[2]: "count" }).sort_values(['count'], ascending=False)

# Initialise figure
plt.figure(figsize = (5,5))
sns.barplot(
    data=disp_df,
    x='region',
    y='count',
    hue='avalanche_Y/N'
)
plt.xticks(
    rotation=45,
    horizontalalignment = 'right',
    fontweight = 'light',
    fontsize = 'large'
)
plt.title('Number of occurances of avalanches per region')
plt.xlabel('REGION', fontsize="15")
plt.ylabel('TOTAL', fontsize="15")
plt.ylabel()
# plt.savefig('Y/N_avalanches_region.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Avalanche_Y/N will not have any nulls as been web scraped.
disp_df = df_n_fresh.groupby('year')[['avalanche_Y/N']].value_counts().reset_index().copy()
disp_df = disp_df.rename(columns={ disp_df.columns[2]: "count" }).sort_values(['count'], ascending=False)

# Initialise figure
plt.figure(figsize = (15,5))
sns.barplot(
    data=disp_df,
    x='year',
    y='count',
    hue='avalanche_Y/N'
)
plt.xticks(
    rotation=45,
    horizontalalignment = 'right',
    fontweight = 'light',
    fontsize = 'large'
)
plt.ylabel('TOTAL', fontsize="15")
plt.xlabel('YEAR', fontsize="15")
plt.title('Number of occurances of avalanches per year', fontsize="15")
# plt.savefig('Y/N_avalanches_region.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Avalanche_Y/N will not have any nulls as been web scraped.
disp_df = df_n_fresh.groupby(['region', 'year'])[['avalanche_Y/N']].value_counts().reset_index().copy()
disp_df = disp_df.rename(columns={ disp_df.columns[2]: "count" })
disp_df = disp_df.rename(columns={ disp_df.columns[3]: "count2" })

condition = disp_df['count'] == 'Yes'
test = disp_df[condition].copy()

# disp_df
figure = plt.figure(figsize=(5,5))
sns.catplot(x='region', y='count2', hue='year', data=test, kind='bar', height=8,aspect=2)
plt.xlabel('REGION', fontsize=17)
plt.xticks(fontsize=15)
plt.ylabel('ACTIVE TOTAL', fontsize=17)
plt.yticks(fontsize=15)
plt.title('avalanche ACTIVE occurances per region per year', fontsize="20")
plt.legend(fontsize="15")
plt.show()


In [ ]:
# Avalanche_Y/N will not have any nulls as been web scraped.
disp_df = df_n_fresh.groupby('month')[['avalanche_Y/N']].value_counts().reset_index().copy()
disp_df = disp_df.rename(columns={ disp_df.columns[2]: "count" })
# disp_df = disp_df.rename(columns={ disp_df.columns[3]: "count2" })

# condition = disp_df['count'] == 'Yes'
# test = disp_df[condition].copy()
# disp_df

# disp_df
figure = plt.figure(figsize=(5,5))
sns.catplot(x='month', y='count', hue='avalanche_Y/N', data=disp_df, kind='bar', height=8,aspect=2)
plt.xlabel('MONTH', fontsize=17)
plt.xticks(fontsize=15)
plt.ylabel('TOTAL', fontsize=17)
plt.yticks(fontsize=15)
plt.title('avalanche occurance per month', fontsize="20")
plt.legend(fontsize="15")
plt.show()


In [ ]:
# Avalanche_Y/N will not have any nulls as been web scraped.
disp_df = df_n_fresh.groupby('snow_condition_per_bigram')[['avalanche_Y/N']].value_counts().reset_index().copy()
disp_df = disp_df.rename(columns={ disp_df.columns[2]: "count" }).sort_values(['count'], ascending=False)

# Initialise figure
plt.figure(figsize = (15,5))
sns.barplot(
    data=disp_df,
    x='snow_condition_per_bigram',
    y='count',
    hue='avalanche_Y/N'
)
plt.xticks(
    rotation=45,
    horizontalalignment = 'right',
    fontweight = 'light',
    fontsize = 'large'
)
plt.ylabel('TOTAL', fontsize="12")
plt.xlabel('SNOW TEXTURE CONDITION', fontsize="12")
plt.title('Number of observational occurances given snow texture')
# plt.savefig('Y/N_avalanches_region.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
## YES

# Avalanche_Y/N will not have any nulls as been web scraped.
disp_df = df_n_fresh.groupby(['region', 'snow_condition_per_bigram'])[['avalanche_Y/N']].value_counts().reset_index().copy()
disp_df = disp_df.rename(columns={ disp_df.columns[2]: "count" })
disp_df = disp_df.rename(columns={ disp_df.columns[3]: "count2" })

condition = ((disp_df['count'] == 'Yes') & (disp_df['region'] == 'Snoqualmie Pass')) | ((disp_df['count'] == 'Yes') & (disp_df['region'] == 'Stevens Pass'))
test = disp_df[condition].sort_values('count2').copy()

# test
figure = plt.figure(figsize=(5,2))
sns.catplot(x='region', y='count2', hue='snow_condition_per_bigram', data=test, kind='bar', height=8,aspect=2)
plt.xticks(fontsize="14")
plt.yticks(fontsize="14")
plt.title('Type of snow texture observations per regions', fontsize="16")
plt.ylabel('TOTAL', fontsize="14")
plt.xlabel('REGION', fontsize="14")
plt.legend(fontsize="12")
plt.show()

## FEATURE ENGINEERING (03)

In [ ]:
def one_to_one_yn(x):
    if x == 'Yes':
        return 1
    else:
        return 0

In [ ]:
# Need to preprocess the snow condition per bigram
def list_to_string(x):
    x = x.strip('[')
    x = x.strip(']')
    x = x.replace("'", "")
    x = x.replace(' ', '')
    x = x.split(',')
    return x

In [ ]:
def feature_eng_dummy_variables_1(df):

    # Apply clean text
    df['snow_condition'] = df['snow_condition_per_bigram'].apply(list_to_string)

    # Get dummy variables aka OHE
    one_hot_df = pd.get_dummies(df['snow_condition'].apply(pd.Series).stack()).sum(level=0)
    df = df.join(one_hot_df)
    one_hot_df = pd.get_dummies(df['zone'])
    df = df.join(one_hot_df)
    one_hot_df = pd.get_dummies(df['observer'])
    df = df.join(one_hot_df)
    one_hot_df = pd.get_dummies(df['month'])
    df = df.join(one_hot_df)
    one_hot_df = pd.get_dummies(df['year'])
    df = df.join(one_hot_df)

    # Delete any potencual copy columns
    df = df.loc[:,~df.columns.duplicated()].copy()

    ## binary encoding for instability and avalancheYN
    df['instability'] = df['instability'].apply(one_to_one_yn)
    df['avalanche_Y/N'] = df['avalanche_Y/N'].apply(one_to_one_yn)

    return df


In [ ]:
## Apply base feature engineering
df_n_fresh = feature_eng_dummy_variables_1(df_n_fresh)
df_n_fresh.columns

In [ ]:
# Save
df_n_fresh.to_csv('df_n_fresh.csv')